In [0]:
#%pip install pgmpy
#dbutils.library.restartPython()

In [0]:
import mlflow
import mlflow.sklearn
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from pgmpy.models import BayesianNetwork

In [0]:
#Cargamos los datos
#df = pd.read_csv('/FileStore/tables/data_proyecto_3/datos_limpio.csv')
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/ma.velandia@uniandes.edu.co/datos_limpio.csv")
df = df.toPandas()
df

,cole_depto_ubicacion,fami_estratovivienda,fami_personashogar,fami_tieneautomovil,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,punt_global
0,VALLE,Estrato 3,9 o más,Si,Si,No,No,Rango de 238.0 a 293.0
1,VALLE,Estrato 5,3 a 4,Si,Si,Si,Si,Rango de 238.0 a 293.0
2,VALLE,Estrato 1,3 a 4,No,No,No,Si,Rango de 48 a 238.0
3,VALLE,Estrato 6,5 a 6,Si,Si,Si,Si,Rango de 293.0 a 337.0
4,CUNDINAMARCA,Estrato 4,3 a 4,Si,Si,Si,Si,Rango de 337.0 a 476
...,...,...,...,...,...,...,...,...
18417,VALLE,Estrato 5,3 a 4,Si,Si,Si,Si,Rango de 337.0 a 476
18418,VALLE,Estrato 4,3 a 4,Si,Si,Si,Si,Rango de 293.0 a 337.0
18419,CAUCA,Estrato 3,5 a 6,No,No,Si,Si,Rango de 48 a 238.0
18420,BOGOTÁ,Estrato 3,7 a 8,Si,Si,Si,Si,Rango de 48 a 238.0


In [0]:
# División de datos en conjuntos de entrenamiento y prueba
#train_data, test_data = train_test_split(df, test_size=0.2, random_state=98)
train_data, test_data = train_test_split(df, test_size=0.3, random_state=90)

# Restricciones
#model = BayesianNetwork([('fami_tieneinternet', 'cole_depto_ubicacion'), ('fami_tieneinternet', 'fami_estratovivienda'), ('fami_tieneinternet', 'fami_tienecomputador'), ('cole_depto_ubicacion', 'fami_tienecomputador'), ('cole_depto_ubicacion', 'punt_global'), ('cole_depto_ubicacion', 'fami_tienelavadora'), ('cole_depto_ubicacion', 'fami_estratovivienda'), ('fami_personashogar', 'cole_depto_ubicacion'), ('fami_personashogar', 'fami_estratovivienda'), ('fami_tienecomputador', 'fami_estratovivienda'), ('fami_tieneautomovil', 'punt_global'), ('fami_tieneautomovil', 'cole_depto_ubicacion'), ('fami_tieneautomovil', 'fami_personashogar'), ('fami_estratovivienda', 'punt_global'), ('fami_estratovivienda', 'fami_tienelavadora'), ('fami_tieneautomovil','fami_estratovivienda')])

#BicScore
#model =BayesianNetwork([('fami_estratovivienda', 'punt_global'), ('fami_estratovivienda', 'fami_tienecomputador'), ('fami_estratovivienda', 'cole_depto_ubicacion'), ('fami_estratovivienda', 'fami_tieneinternet'), ('fami_personashogar', 'fami_tieneautomovil'), ('fami_tieneautomovil', 'fami_estratovivienda'), ('fami_tieneautomovil', 'fami_tienecomputador'), ('fami_tieneautomovil', 'fami_tienelavadora'), ('fami_tieneautomovil', 'punt_global'), ('fami_tienecomputador', 'fami_tieneinternet'), ('fami_tienecomputador', 'fami_tienelavadora'), ('fami_tienecomputador', 'punt_global'), ('fami_tieneinternet', 'fami_tienelavadora')])

#k2score
model =BayesianNetwork([('cole_depto_ubicacion', 'punt_global'), ('fami_estratovivienda', 'punt_global'), ('fami_estratovivienda', 'fami_tienecomputador'), ('fami_estratovivienda', 'cole_depto_ubicacion'), ('fami_estratovivienda', 'fami_tieneinternet'), ('fami_estratovivienda', 'fami_tienelavadora'), ('fami_personashogar', 'fami_tieneautomovil'), ('fami_personashogar', 'fami_estratovivienda'), ('fami_tieneautomovil', 'fami_estratovivienda'), ('fami_tieneautomovil', 'fami_tienecomputador'), ('fami_tieneautomovil', 'fami_tienelavadora'), ('fami_tieneautomovil', 'punt_global'), ('fami_tieneautomovil', 'cole_depto_ubicacion'), ('fami_tienecomputador', 'fami_tieneinternet'), ('fami_tienecomputador', 'fami_tienelavadora'), ('fami_tienelavadora', 'fami_tieneinternet'), ('punt_global', 'fami_tienecomputador')])



model.fit(train_data, estimator=MaximumLikelihoodEstimator)  # Entrena el modelo con MLE

# registre el experimento
#mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
mlflow.set_experiment("/Users/ma.velandia@uniandes.edu.co/proyecto_experiments_3")
#mlflow.set_experiment("proyecto_experiments_3")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2513764676775162', creation_time=1701656537853, experiment_id='2513764676775162', last_update_time=1701660055698, lifecycle_stage='active', name='/Users/ma.velandia@uniandes.edu.co/proyecto_experiments_3', tags={'mlflow.experiment.sourceName': '/Users/ma.velandia@uniandes.edu.co/proyecto_experiments_3',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'ma.velandia@uniandes.edu.co',
 'mlflow.ownerId': '5334649168356220'}>

In [0]:

run_name='k2score_2'
# Inicia un nuevo run de MLflow
with mlflow.start_run(run_name=run_name):

    # Realiza predicciones en el conjunto de prueba
    predictions = model.predict(test_data.drop(columns=['punt_global']))

    # Calcula y registra la precisión
    accuracy = accuracy_score(test_data['punt_global'], predictions['punt_global'])
    mlflow.log_metric("accuracy", accuracy)

    # Calcula y registra la sensibilidad (recall)
    recall = recall_score(test_data['punt_global'], predictions['punt_global'],average='weighted')
    mlflow.log_metric("recall", recall)

    # Calcula y registra la precisión
    precision = precision_score(test_data['punt_global'], predictions['punt_global'],average='weighted')
    mlflow.log_metric("precision", precision)

    # Calcula y registra el valor F1
    f1 = f1_score(test_data['punt_global'], predictions['punt_global'],average='weighted')
    mlflow.log_metric("f1", f1)

    # Calcula y registra la matriz de confusión
    #conf_matrix = confusion_matrix(test_data['punt_global'], predictions['punt_global'])
    #mlflow.log_artifact(pd.DataFrame(conf_matrix), "confusion_matrix.csv")

    # Guarda el modelo en MLflow
    mlflow.sklearn.log_model(model, "bayesian-model")



  0%|          | 0/1147 [00:00<?, ?it/s]

2023/12/04 03:24:07 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]